In [16]:
%pip -q install google-genai

In [17]:
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [18]:
from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [20]:
!pip install -q google-adk

In [32]:
from datetime import date
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
import textwrap
from IPython.display import display, Markdown, HTML
import requests
import warnings
import time

In [41]:
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [42]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [47]:
MSG_BOAS_VINDAS = "Opa, bateu a fome? Bora agilizar o rango!"
MSG_INTRO_INGREDIENTES = "Me conta quais ingredientes você tem em casa?"
MSG_PEDIR_INGREDIENTES = "Ingredientes disponíveis (ou digite 'sair' para encerrar):"
MSG_SAIR = "Entendido. Saindo. Até a próxima!"
MSG_SEM_INGREDIENTES = "Tem nada mesmo? Que pena. Espero que a padoca da esquina esteja aberta."
MSG_SUCESSO_INGREDIENTES = "Massa! Vamos ver o que dá pra fazer..."
MSG_SEM_OPCOES = "Eita, deu ruim aqui. Não encontrei nada que dá pra fazer com os ingredientes informados. Que tal tentar com ingredientes diferentes?"
MSG_PEDIR_RECEITA = "Qual das receitas acima você escolhe? (Copie o nome ou o número, ou digite 'sair'):"
MSG_SEM_RECEITA_ESCOLHIDA = "Entendido. Nenhuma receita agradou."
MSG_FINAL = "Fechou, né?"
MSG_PEDIR_SIM_NAO = "Você tem todos esses ingredientes? (sim/não):"
MSG_RESPOSTA_INVALIDA_SIM_NAO = "Responda apenas 'sim' ou 'não'."
MSG_FALTA_INGREDIENTES = "Entendido. Que tal tentarmos encontrar outras opções com os ingredientes que você tem?"

QUESTION_PROMPT_EXPECTED = "Você tem todos esses ingredientes?"

In [48]:
def to_markdown(text):
    """Converte texto para Markdown, tratando marcadores de lista e indentação."""
    text = str(text)
    text = text.replace('•', '  *')
    return Markdown(text)

def call_agent(agent: Agent, message_text: str) -> str:
    """Chama um agente ADK com uma string de input e retorna a resposta final como string."""
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)]) # Usa a string de input aqui

    final_response = ""
    try:
        for event in runner.run(user_id="user1", session_id="session1", new_message=content):
            if event.is_final_response():
                for part in event.content.parts:
                    if part.text is not None:
                        final_response += part.text

    except Exception as e:
         print(f"Erro durante a execução do runner para o agente {agent.name}: {e}")
         raise e
    return final_response.strip()


# --- Definição dos agentes --- #

##########################################
# --- Agente 1: Consultor de Geladeira e Dispensa --- #
##########################################

def agente_consultor(message_text: str):
    """
    Consulta receitas com base nos ingredientes disponíveis do usuário.
    Input (str): String contendo os ingredientes e data (ex: "Ingredientes: ovo, farinha\nData: 17/05/2025").
    Output: str = Lista numerada (Markdown) das 3 receitas sugeridas (Nome: Descrição).
    """
    consultor = Agent(
        name="agente_consultor",
        model="gemini-2.0-flash",
        description="Agente que busca receitas com base nos ingredientes fornecidos.",
        tools=[google_search],
        instruction="""
        Você é o agente que irá iniciar a conversa com o usuário, ajudando-o a encontrar receitas com o que ele tem em casa.
        Seja direto, sucinto e prático em suas respostas.
        O usuário te deu uma string no input contendo os ingredientes e a data de hoje.
        Sua tarefa é:
        (1) Analisar o input do usuário para identificar os ingredientes disponíveis.
        (2) Utilizar sua ferramenta (Google Search) para encontrar receitas que utilizem **esses ingredientes listados no input**.
        (3) Selecionar as 3 receitas mais relevantes e bem avaliadas encontradas.
        (4) Apresentar essas 3 receitas ao usuário em formato de **lista numerada (Markdown)**, incluindo apenas o **nome da receita** e uma **descrição MUITO breve** para cada uma. Não inclua ingredientes ou passo a passo aqui.
        Exemplo de formato de saída:
        1. Omelete Simples: Ótima para um café rápido.
        2. Arroz de Forno: Prato único e delicioso.
        3. Sopa de Legumes: Conforto em dias frios.
        """
    )

    print(f"Vamos ver o que dá pra fazer com isso...")
    ingredientes_sugeridos = call_agent(consultor, message_text)
    return ingredientes_sugeridos


##########################################
# --- Agente 2: Agente Intermediário --- #
##########################################

def agente_intermediario(message_text: str):
    """
    Lista os ingredientes para a receita escolhida e pergunta ao usuário se ele os tem.
    Input (str): String contendo o nome da receita escolhida e as sugestões anteriores (ex: "Receita: Omelete Simples\nSugestões: 1. Omelete...\n...").
    Output: str = Lista dos ingredientes necessários + a pergunta exata "Você tem todos esses ingredientes?".
    """
    intermediario = Agent(
        name="agente_intermediario",
        model="gemini-2.0-flash",
        description="Agente que lista ingredientes para uma receita específica e faz a pergunta de checagem.",
        tools=[google_search],
        instruction="""
        Você recebeu uma string no input contendo o nome da receita que o usuário escolheu e, para contexto, a lista original de sugestões do agente anterior.
        Sua tarefa é:
        (1) Identificar o **nome da receita escolhida** na string de input.
        (2) Utilizar sua ferramenta (Google Search) para encontrar a lista COMPLETA de ingredientes necessária para **essa receita escolhida**.
        (3) Apresentar a lista de ingredientes de forma clara (pode ser em lista, como no exemplo).
        (4) Imediatamente após a lista de ingredientes, inclua **EXATAMENTE** a seguinte frase: "Você tem todos esses ingredientes?"
        Não adicione nenhuma outra frase ou explicação após a pergunta.

        Exemplo de formato de saída:
        Para fazer [Nome da Receita Escolhida], você vai precisar de:
        - 4 ovos
        - 500ml de leite
        - 200g farinha de trigo
        - Açúcar a gosto

        Você tem todos esses ingredientes?
        """
    )

    print(f"Agente Intermediário: Processando input para verificar ingredientes...")
    lista_ingredientes_e_pergunta = call_agent(intermediario, message_text)
    return lista_ingredientes_e_pergunta


##########################################
# --- Agente 3: Agente Executor --- #
##########################################


def agente_executor(message_text: str):
    """
    Fornece a receita completa (ingredientes e passo a passo).
    Este agente só é chamado se o usuário confirmar que tem os ingredientes.
    Input (str): String contendo o nome da receita escolhida e a lista de ingredientes verificada anteriormente (ex: "Receita: Omelete Simples\nIngredientes Verificados:\n- 4 ovos\n...").
    Output: str = Receita completa (ingredientes e passo a passo).
    """
    executor = Agent(
        name="agente_executor",
        model="gemini-2.0-flash",
        description="Agente que fornece a receita completa.",
        tools=[google_search],
        instruction="""
        Você recebeu uma string no input contendo o nome de uma receita que o usuário escolheu e a lista de ingredientes que foi checada anteriormente.
        Sua missão é fornecer a receita COMPLETA para **essa receita identificada no input**. Inclua a lista de ingredientes (pode reafirmar a lista do input ou buscar a sua própria versão completa) e o passo a passo sucinto para prepará-la.
        Formate a receita de forma clara, talvez usando Markdown para títulos e listas.
        """
    )


    print(f"Quase lá! Logo logo retorno com a receita completa.")
    receita_completa = call_agent(executor, message_text)
    return receita_completa


# --- Código de Orquestração --- #


def main():
    """Orquestra a interação entre os agentes e o usuário."""

    data_de_hoje = date.today().strftime("%d/%m/%Y")

    display(HTML(f"<h1>{MSG_BOAS_VINDAS}</h1>"))
    print(MSG_INTRO_INGREDIENTES)
    time.sleep(1)

    # --- Loop Principal --- #
    while True:
        # 1. Input Inicial de Ingredientes
        ingredientes_input = get_non_empty_input(MSG_PEDIR_INGREDIENTES + " ", MSG_SEM_INGREDIENTES, exit_command='sair')

        if ingredientes_input.lower() == 'sair':
            print(MSG_SAIR)
            break

        if not ingredientes_input:
            print(MSG_SEM_INGREDIENTES)
            continue

        print(MSG_SUCESSO_INGREDIENTES)
        time.sleep(1)

        # 2. Chamar Agente Consultor
        display_section_header("Receitas Sugeridas")
        try:

            input_para_consultor = f"Ingredientes disponíveis: {ingredientes_input}\nData de hoje: {data_de_hoje}"
            resultado_consultor = agente_consultor(input_para_consultor) # Passa APENAS a string
            display_agent_output(resultado_consultor)

            if not resultado_consultor or (isinstance(resultado_consultor, str) and not resultado_consultor.strip()):
                 print(MSG_SEM_OPCOES)
                 continue

        except Exception as e:
            print(f"Ocorreu um erro ao consultar as receitas: {e}")
            print("Por favor, tente novamente.")
            continue

        # 3. Input de Escolha da Receita
        print("\n")
        receita_input = get_non_empty_input(MSG_PEDIR_RECEITA + " ", MSG_SEM_RECEITA_ESCOLHIDA, exit_command='sair')

        if receita_input.lower() == 'sair':
            print(MSG_SAIR)
            break

        if not receita_input:
            print(MSG_SEM_RECEITA_ESCOLHIDA)
            continue

        # 4. Chamar Agente Intermediário
        display_section_header(f"Verificando ingredientes para: '{receita_input}'")
        try:
            # Prepara a ÚNICA string de input para o Agente 2
            input_para_intermediario = f"Receita escolhida: {receita_input}\nSugestões anteriores: {resultado_consultor}"
            resultado_intermediario = agente_intermediario(input_para_intermediario) # Passa APENAS a string
            display_agent_output(resultado_intermediario)

        except Exception as e:
            print(f"Ocorreu um erro ao verificar os ingredientes: {e}")
            print("Por favor, tente novamente.")
            continue

        # 5. Verificar Saída do Agente Intermediário e Pedir Input SIM/NÃO

        if isinstance(resultado_intermediario, str) and QUESTION_PROMPT_EXPECTED.strip() in resultado_intermediario:
             print("\n")
             time.sleep(1)

             resposta_sim_nao = ""
             while resposta_sim_nao.lower() not in ['sim', 'não', 'nao']:
                 resposta_sim_nao = get_non_empty_input(MSG_PEDIR_SIM_NAO + " ", MSG_RESPOSTA_INVALIDA_SIM_NAO).lower()
                 if resposta_sim_nao not in ['sim', 'não', 'nao']:
                      print(MSG_RESPOSTA_INVALIDA_SIM_NAO)

             # 6. Ramificação baseada na resposta SIM/NÃO
             if resposta_sim_nao.lower() in ['sim', 's']:
                 # Usuário tem os ingredientes, chamar Agente Executor
                 display_section_header(f"Preparando a receita de '{receita_input}'")
                 try:

                      input_para_executor = f"Receita escolhida: {receita_input}\nIngredientes verificados: {resultado_intermediario}"
                      resultado_executor = agente_executor(input_para_executor)
                      display_agent_output(resultado_executor)

                      # Processo de receita concluído com sucesso
                      print("\n")
                      display(HTML(f"<h2>{MSG_FINAL} Bom apetite!</h2>"))
                      break # Sai do loop principal

                 except Exception as e:
                      print(f"Ocorreu um erro ao obter a receita completa: {e}")
                      print("Por favor, tente novamente com a mesma ou outra receita.")
                      continue # Volta para o início do loop

             else: # resposta_sim_nao é 'não' ou 'nao'
                 # Usuário não tem os ingredientes
                 print("\n")
                 print(MSG_FALTA_INGREDIENTES)
                 time.sleep(2)
                 continue # Volta para o início do loop

        else:
            # Se a saída do agente 2 NÃO foi a pergunta esperada
            print("\n")
            print("Eita, parece que deu ruim.")
            print("Tentando outra receita ou ingredientes...")
            continue # Volta para o início do loop


    # --- Fim do Loop Principal ---
    pass

# --- Executa a função principal ---
if __name__ == "__main__":
    main()

Me conta quais ingredientes você tem em casa?
Ingredientes disponíveis (ou digite 'sair' para encerrar): ovo
Massa! Vamos ver o que dá pra fazer...


Vamos ver o que dá pra fazer com isso...


Olá! Com ovo, posso te sugerir algumas receitas rápidas e fáceis:

1.  **Omelete Simples:** Uma opção rápida e versátil para qualquer refeição.
2.  **Ovos Mexidos:** Clássico e fácil, perfeito para um café da manhã rápido.
3.  **Ovos à Parmegiana:** Uma forma saborosa e diferente de preparar os ovos.



Qual das receitas acima você escolhe? (Copie o nome ou o número, ou digite 'sair'): Omelete Simples


Agente Intermediário: Processando input para verificar ingredientes...


Para fazer Omelete Simples, você vai precisar de:
- 2 ou 3 ovos
- Sal a gosto
- Pimenta do reino a gosto
- 1 colher de óleo ou azeite ou manteiga
- Recheios opcionais como queijo, presunto, tomate, cebola, cheiro-verde
- 1 colher de farinha de trigo (opcional)
- Leite ou água (opcional)

Você tem todos esses ingredientes?



Você tem todos esses ingredientes? (sim/não): sim


Quase lá! Logo logo retorno com a receita completa.


## Omelete Simples Completa

Aqui está uma receita completa de omelete simples, com algumas opções e dicas para você personalizar:

### Ingredientes:

*   2 ou 3 ovos grandes
*   Sal a gosto
*   Pimenta do reino a gosto
*   1 colher de sopa de óleo, azeite ou manteiga
*   Recheios opcionais: queijo (mussarela, prato, parmesão, etc.), presunto picado, tomate picado, cebola picada, ervas frescas picadas (cheiro-verde, orégano, manjericão)
*   1 colher de chá de farinha de trigo (opcional, para uma omelete mais fofinha)
*   1 colher de sopa de leite ou água (opcional, para deixar a omelete mais leve)

### Modo de Preparo:

1.  **Prepare os ingredientes:** Pique os recheios que você escolheu e deixe-os separados.
2.  **Bata os ovos:** Em uma tigela, quebre os ovos e bata-os com um garfo ou batedor de arame até que as gemas e claras estejam bem misturadas. Adicione o sal, a pimenta do reino e, se desejar, a farinha de trigo e o leite ou água. Bata até incorporar todos os ingredientes.
3.  **Aqueça a frigideira:** Coloque a frigideira em fogo médio e adicione o óleo, azeite ou manteiga. Espere aquecer bem.
4.  **Despeje os ovos:** Despeje a mistura de ovos na frigideira quente.
5.  **Cozinhe a omelete:** Deixe cozinhar por alguns segundos, até que a parte de baixo comece a firmar.
6.  **Adicione o recheio:** Se estiver usando recheios, espalhe-os sobre metade da omelete.
7.  **Dobre a omelete:** Com uma espátula, dobre a omelete ao meio, cobrindo o recheio.
8.  **Finalize a cocção:** Cozinhe por mais alguns segundos, até que o queijo derreta (se estiver usando) e a omelete esteja dourada dos dois lados.
9.  **Sirva:** Desligue o fogo, transfira a omelete para um prato e sirva imediatamente.

**Dicas:**

*   Para uma omelete mais cremosa, cozinhe em fogo baixo e mexa delicadamente os ovos na frigideira antes de adicionar o recheio.
*   Não coloque recheio em excesso para não dificultar na hora de dobrar a omelete.
*   Você pode usar outros temperos de sua preferência, como orégano, salsa desidratada, cebola em pó, alho em pó, etc.
*   Experimente diferentes combinações de recheios para descobrir seus sabores favoritos!